In [1]:
import datetime
import glob
import os

import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import scipy.stats as stats
import tensorflow as tf
from tqdm import tqdm

import utils

In [2]:
df = pd.read_csv('merged_df_norm.csv',parse_dates=['time'])
df = df.set_index('time')

In [3]:
(train_df,val_df,test_df), num_features = utils.train_test_val_split(df)

In [4]:
day_list = []
for group in df.groupby(df.index.date):
    if (group[1]['weekday'].astype(int).values[0]==1) & (group[1]['tp'][group[1]['sf']>0].any()):
        day_list.append(group[1])

In [5]:
dense_no = tf.keras.models.load_model('no_weather')
dense_wx = tf.keras.models.load_model('weather')

In [6]:
tem = np.ones(len(day_list))

width = 72
for ind,day in enumerate(day_list):
    day = day.drop(['sf', 'tp', 'tp_acc', 'sf_acc',],axis=1)
    if len(day)>240:
        pred = dense_no.predict(day[14*12:14*12+width])
        val = day[14*12:14*12+width]
        tem[ind] = np.sqrt(((val['speed'].values-pred.T)**2).mean())
    
print(tem.mean())

0.9201060166323017


In [7]:
tem = np.ones(len(day_list))

width = 72
for ind,day in enumerate(day_list):
    if len(day)>240:
        pred = dense_wx.predict(day[14*12:14*12+width])
        val = day[14*12:14*12+width]
        tem[ind] = np.sqrt(((val['speed'].values-pred.T)**2).mean())
    
print(tem.mean())

0.9353755228568565


In [8]:
ind_0 = np.random.randint(0,len(df)-1)
width = 72
ticks = '1H'

prediction = dense.predict(df[ind_0:ind_0+width])

plt.scatter(time[ind_0:ind_0+3*width],df[ind_0:ind_0+3*width]['speed'].values,label='observed')

plt.scatter(time[ind_0+width:ind_0+2*width],prediction,label='prediction')

plt.grid()
start = time.loc[ind_0]
end = time.loc[ind_0+3*width]
dates = pd.date_range(start=start,end=end,freq=ticks)
plt.xticks(ticks = dates,labels = dates,rotation=45,ha='right',size=10)
plt.legend()

NameError: name 'dense' is not defined